In [ ]:
#import the the extracted datas and merge
library(Seurat) #preinstalled Seurat 4.4.0 ,SeuratObject 4.1.4
library(dplyr)
library(Matrix) #Matrix 1.6.1.1
CP1<-Read10X(data.dir="./CP1")
CP2<-Read10X(data.dir="./CP2")
CP3<-Read10X(data.dir="./CP3")
Ctrl1<-Read10X(data.dir="./Ctrl1")
Ctrl2<-Read10X(data.dir="./Ctrl2")
Ctrl3<-Read10X(data.dir="./Ctrl3")

Object_CP1<-CreateSeuratObject(counts=CP1,projects='MouseCP1',min.cells=3,min.features=200)
Object_CP2<-CreateSeuratObject(counts=CP2,projects='MouseCP2',min.cells=3,min.features=200)
Object_CP3<-CreateSeuratObject(counts=CP3,projects='MouseCP3',min.cells=3,min.features=200)
Object_CP1[["percent_mt"]]<-PercentageFeatureSet(Object_CP1,pattern="^mt-")
Object_CP2[["percent_mt"]]<-PercentageFeatureSet(Object_CP2,pattern="^mt-")
Object_CP3[["percent_mt"]]<-PercentageFeatureSet(Object_CP3,pattern="^mt-")

Object_Ctrl1<-CreateSeuratObject(counts=Ctrl1,projects='MouseCtrl1',min.cells=3,min.features=200)
Object_Ctrl2<-CreateSeuratObject(counts=Ctrl2,projects='MouseCtrl2',min.cells=3,min.features=200)
Object_Ctrl3<-CreateSeuratObject(counts=Ctrl3,projects='MouseCtrl3',min.cells=3,min.features=200)
Object_Ctrl1[["percent_mt"]]<-PercentageFeatureSet(Object_Ctrl1,pattern="^mt-")
Object_Ctrl2[["percent_mt"]]<-PercentageFeatureSet(Object_Ctrl2,pattern="^mt-")
Object_Ctrl3[["percent_mt"]]<-PercentageFeatureSet(Object_Ctrl3,pattern="^mt-")

**Rename the elements in Orig.ident column as CPX/Ctrl**

In [ ]:
Object_CP1@meta.data <- Object_CP1@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "CP1", orig.ident))
Object_CP2@meta.data <- Object_CP2@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "CP2", orig.ident))
Object_CP3@meta.data <- Object_CP3@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "CP3", orig.ident))

Object_Ctrl1@meta.data <- Object_Ctrl1@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "Ctrl1", orig.ident))
Object_Ctrl2@meta.data <- Object_Ctrl2@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "Ctrl2", orig.ident))
Object_Ctrl3@meta.data <- Object_Ctrl3@meta.data %>%
 dplyr::mutate(orig.ident = ifelse(orig.ident == "SeuratProject", "Ctrl3", orig.ident))

**Refine the data according to your requirements**

In [ ]:

Sub_CP1<-subset(Object_CP1,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)
Sub_CP2<-subset(Object_CP2,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)
Sub_CP3<-subset(Object_CP3,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)

Sub_Ctrl1<-subset(Object_Ctrl1,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)
Sub_Ctrl2<-subset(Object_Ctrl2,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)
Sub_Ctrl3<-subset(Object_Ctrl3,subset=nFeature_RNA>200&nFeature_RNA<2500&percent_mt<5)

**Merge the repeats into one**

In [ ]:
Merged_CP<- merge(Sub_CP1, y = list(Sub_CP2, Sub_CP3), add.cell.ids = c('CP1', 'CP2', 'CP3'))

Merged_Ctrl<- merge(Sub_Ctrl1, y = list(Sub_Ctrl2, Sub_Ctrl3), add.cell.ids = c('Ctrl1', 'Ctrl2', 'Ctrl3'))


In [ ]:
#QC
plot1 <- FeatureScatter(Merged_CP, feature1 = "nCount_RNA", feature2 = "percent_mt")
plot2 <- FeatureScatter(Merged_CP, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

plot3 <- FeatureScatter(Merged_Ctrl, feature1 = "nCount_RNA", feature2 = "percent_mt")
plot4 <- FeatureScatter(Merged_Ctrl, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot3 + plot4


**High variable features**

In [ ]:

Merged_CP <- FindVariableFeatures(Merged_CP,selection_method="vst",nfeatures=2000)
top30 <- head(VariableFeatures(Merged_CP), 30)
plot5 <- VariableFeaturePlot(Merged_CP)
plot6 <- LabelPoints(plot = plot5, points = top30, repel = TRUE)
plot5 + plot6

Merged_Ctrl <- FindVariableFeatures(Merged_Ctrl,selection_method="vst",nfeatures=2000)
top20 <- head(VariableFeatures(Merged_Ctrl), 30)
plot7 <- VariableFeaturePlot(Merged_Ctrl)
plot8 <- LabelPoints(plot = plot7, points = top30, repel = TRUE)
plot7 + plot8

**Scale the data(ignore the overlap error when occurs)**

In [ ]:

all.genes_CP <- rownames(Merged_CP)
Merged_CP <- ScaleData(Merged_CP, features = all.genes_CP)
Merged_CP<-RunPCA(Merged_CP,features=VariableFeatures(object = Merged_CP))

all.genes_Ctrl <- rownames(Merged_Ctrl)
Merged_Ctrl <- ScaleData(Merged_Ctrl, features = all.genes_Ctrl)
Merged_Ctrl<-RunPCA(Merged_Ctrl,features=VariableFeatures(object = Merged_Ctrl))

**Cluster the cells**

In [ ]:
Merged_CP_Clustering <- FindNeighbors(Merged_CP, dims = 1:13)
Merged_CP_Clustered <- FindClusters(Merged_CP_Clustering, resolution = 0.5)

Merged_Ctrl_Clustering <- FindNeighbors(Merged_Ctrl, dims = 1:13)
Merged_Ctrl_Clustered <- FindClusters(Merged_Ctrl_Clustering, resolution = 0.5)

**Query and match to the genesets**

In [ ]:
Merged_CP.markers <- FindAllMarkers(Merged_CP_Clustered, only.pos = TRUE)
Merged_CP.markers %>%
    group_by(cluster) %>%
    dplyr::filter(avg_log2FC > 1)
    
Merged_Ctrl.markers <- FindAllMarkers(Merged_Ctrl_Clustered, only.pos = TRUE)
Merged_Ctrl.markers %>%
    group_by(cluster) %>%
    dplyr::filter(avg_log2FC > 1)

**Annatation**

In [ ]:
library(SingleR)
library(celldex)
library(pheatmap)

#turning Seurat matrix into SingleR matrix
SingleR_CP<-GetAssayData(Merged_CP_Clustered,slot="data")
SingleR_Ctrl<-GetAssayData(Merged_Ctrl_Clustered,slot="data")
#import mouse gene marker library manually(cuz automated failed)
MouseRNAseq<-readRDS("MouseRNAseqData.rds") 

#general match to one dataset
Merged_CP_Clustered.match<-SingleR(test=SingleR_CP,ref=MouseRNAseq,labels=MouseRNAseq$label.main)
Merged_CP_Clustered.match

Merged_Ctrl_Clustered.match<-SingleR(test=SingleR_Ctrl,ref=MouseRNAseq,labels=MouseRNAseq$label.main)
Merged_Ctrl_Clustered.match

#SingleR table PCA and define the labels
table(Merged_CP_Clustered.match$labels,Merged_CP_Clustered$seurat_clusters)

table(Merged_CP_Clustered.match$labels,Merged_CP_Clustered$seurat_clusters)

Merged_CP_Clustered@meta.data$labels <-Merged_CP_Clustered.match$labels
Merged_Ctrl_Clustered@meta.data$labels <-Merged_Ctrl_Clustered.match$labels

**Plot the de-dims**

In [ ]:
#PCA
plot_CP_PCA<-DimPlot(Merged_CP_Clustered,group.by=c("labels"),reduction="pca")
plot_Ctrl_PCA<-DimPlot((Merged_Ctrl_Clustered),group.by=c("labels"),reduction="pca",dims=c(2,1))#dims=c(2,1),rotate the plot because unexplainale data appearance
plot_CP_PCA + plot_Ctrl_PCA

#TSNE
Merged_CP_Clustered_TSNE <- RunTSNE(Merged_CP_Clustered, dims = 1:13)
plot_CP_TSNE<-DimPlot(Merged_CP_Clustered_TSNE, group.by=c("labels"),reduction = "tsne")

Merged_Ctrl_Clustered_TSNE <- RunTSNE(Merged_Ctrl_Clustered, dims = 1:13)
plot_Ctrl_TSNE<-DimPlot(Merged_Ctrl_Clustered_TSNE, group.by=c("labels"),reduction = "tsne",dims=c(2,1))
plot_CP_TSNE + plot_Ctrl_TSNE

#UMAP unbale to comply because as_cholmod_sparse not provided by package "Matrix")
#Merged_CP_Clustered_UMAP <- RunUMAP(Merged_CP_Clustered, dims = 1:13)
#plot_CP_UMAP<-DimPlot(Merged_CP_Clustered_UMAP, group.by=c("labels"),reduction = "umap")

#Merged_Ctrl_Clustered_UMAP <- RunUMAP(Merged_Ctrl_Clustered, dims = 1:13)
#plot_Ctrl_UMAP<-DimPlot(Merged_Ctrl_Clustered_UMAP, group.by=c("Ctrl"),reduction = "tsne",dims=c(2,1))
#plot_CP_UMAP + plot_Ctrl_UMAP